In [11]:
import sagemaker
import os
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

In [8]:
role = get_execution_role()
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

In [9]:
model_img = sagemaker.image_uris.retrieve("sklearn", region, "1.2-1") # This function returns path to latest docker image
print(model_img)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


141502667606.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3


In [12]:
os.makedirs("01_simple_processing", exist_ok=True) # Create folder for training code

In [13]:
%%writefile 01_simple_processing/create_text_file.py
from __future__ import print_function
import boto3
from botocore.exceptions import NoCredentialsError

if __name__ == '__main__':

    def upload_to_s3(bucket_name, file_name, content):
        # Create an S3 client
        s3 = boto3.client('s3')
    
        try:
            # Upload the file
            s3.put_object(Bucket=bucket_name, Key=file_name, Body=content)
            print(f"File '{file_name}' uploaded to bucket '{bucket_name}'.")
        except NoCredentialsError:
            print("Credentials not available.")
    
    # Define your bucket name, file name, and content
    bucket_name = 'sagemaker-bucket-ds'
    file_name = '01_example.txt'
    content = 'Hello, this is a test file.'
    
    # Upload the file
    upload_to_s3(bucket_name, file_name, content)

Writing 01_simple_processing/create_text_file.py


In [14]:
!aws s3 cp 01_simple_processing/create_text_file.py s3://sagemaker-bucket-ds/PROCESSING/01_CODE/

upload: 01_simple_processing/create_text_file.py to s3://sagemaker-bucket-ds/PROCESSING/01_CODE/create_text_file.py


In [15]:
# Define the ScriptProcessor
script_processor = ScriptProcessor(
    role=role,
    image_uri=model_img,
    command=['python3'],
    instance_count=1,
    instance_type='ml.t3.medium',
    volume_size_in_gb=5,
    max_runtime_in_seconds=3600,
    base_job_name='01-processing-job',
    sagemaker_session=sagemaker_session
)

In [16]:
# Define the Processing Inputs and Outputs
processing_inputs = []

processing_outputs = []



In [ ]:
# Run the Processing Job
script_processor.run(
    code='s3://sagemaker-bucket-ds/PROCESSING/01_CODE/create_text_file.py',
    inputs=processing_inputs,
    outputs=processing_outputs
)

INFO:sagemaker:Creating processing-job with name 01-processing-job-2024-07-10-14-06-54-739


....